In [ ]:
# Install necessary libraries (uncomment if needed)
# !pip install opencv-python numpy matplotlib scikit-learn

# Import libraries
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Load FER2013 dataset
# Ensure the dataset path is correct
data = pd.read_csv('/content/drive/MyDrive/fer2013.csv')

# Inspect the dataset
print("Dataset Head:")
print(data.head())

# Check dataset structure
print("\nDataset Info:")
print(data.info())

# Extract the images and labels
X = []
y = []

# Convert pixel data to 2D arrays and extract corresponding emotion labels
for index, row in data.iterrows():
    # Convert the string of pixel values into a numpy array
    pixels = np.array(row['pixels'].split(), dtype=np.float32)
    pixels = pixels.reshape(48, 48)  # Reshaping to 48x48 image
    X.append(pixels)
    y.append(row['emotion'])

X = np.array(X)
y = np.array(y)

# Normalize the image data
X = X / 255.0  # Scaling pixel values between 0 and 1

# Flatten the images into 1D arrays
X = X.reshape(X.shape[0], -1)

# Label Encoding for emotion labels
le = LabelEncoder()
y = le.fit_transform(y)  # Convert emotion labels into integers

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model with a linear kernel
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'\nAccuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(le.classes_))
plt.xticks(tick_marks, le.classes_, rotation=45)
plt.yticks(tick_marks, le.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

# Example: Using Histogram of Oriented Gradients (HOG) for feature extraction
fd, hog_image = hog(X_train[0].reshape(48, 48),
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=True,
                    channel_axis=None)
plt.figure()
plt.imshow(hog_image, cmap=plt.cm.gray)
plt.title('HOG Features')
plt.show()

